In [175]:
import cv2
import mediapipe as mp
import time
import pandas as pd
import numpy as np

In [176]:
num_coords = 501

In [177]:
def landmmark_work(results, mpPose, mpDraw,mpHol, frame):
    if (results.pose_landmarks):
        mpDraw.draw_landmarks(frame, results.face_landmarks, mpHol.FACEMESH_TESSELATION, 
                                 mpDraw.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
                                 mpDraw.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                                 )
        
        # 2. Right hand
        mpDraw.draw_landmarks(frame, results.right_hand_landmarks, mpHol.HAND_CONNECTIONS, 
                                 mpDraw.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                 mpDraw.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                 )

        # 3. Left Hand
        mpDraw.draw_landmarks(frame, results.left_hand_landmarks, mpHol.HAND_CONNECTIONS, 
                                 mpDraw.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                 mpDraw.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                 )

        # 4. Pose Detections
        mpDraw.draw_landmarks(frame, results.pose_landmarks, mpHol.POSE_CONNECTIONS, 
                                 mpDraw.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mpDraw.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )

def request_capture(type=0, path=None):

    if(type):
        capture = cv2.VideoCapture(path)
    else:
        capture = cv2.VideoCapture(0)
    return capture

In [178]:
mpPose = mp.solutions.pose
mpDraw = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [179]:


from cv2 import CAP_PROP_APERTURE


holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
#0 for webcam
#1 for default video
path_to_file = 'balad.mp4'
capture = request_capture(1, path_to_file)

# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH) + 0.5)
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT) + 0.5)
#size = (width, height)
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('your_video.avi', fourcc, 20.0, size)
pTime = 0

while(1):
    ret , frame = capture.read()
    try:
        #mpPose processa cada frame no formato RGB
        frameRGB = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
        results = holistic.process(frameRGB)

        #R_landmark = results.pose_landmarks

        cTime = time.time()
        
        landmmark_work(results, mpPose, mpDraw, mp_holistic, frame)
        
        cv2.imshow("Video", frame)
        out.write(frame)
    except:
        break

    k = cv2.waitKey(5) & 0xff
    if k == 27:
        break

frame_width = int(capture.get(3))
frame_height = int(capture.get(4))
# Define the codec and create VideoWriter object.The output is stored in 'outpy.avi' file.
capture.release()
out.release()
cv2.destroyAllWindows()

Export Landmarks

In [180]:
import csv
import os
import numpy as np

In [ ]:
colunas = ['x', 'y', 'z', 'v']

In [ ]:
'''df = pd.DataFrame()
for i in range(1,num_coords):
    data = {"x": results.pose_landmarks.landmark[i].x, "y":results.pose_landmarks.landmark[i].y, "z":results.pose_landmarks.landmark[i].z, "v":results.pose_landmarks.landmark[i].visibility}
    df = df.append(data, ignore_index=True)'''

'df = pd.DataFrame()\nfor i in range(1,num_coords):\n    data = {"x": results.pose_landmarks.landmark[i].x, "y":results.pose_landmarks.landmark[i].y, "z":results.pose_landmarks.landmark[i].z, "v":results.pose_landmarks.landmark[i].visibility}\n    df = df.append(data, ignore_index=True)'

In [ ]:
filename = 'coord.csv'

In [ ]:
landmarks_class = ['class']
for i in range (1, num_coords+1):
    landmarks_class += ['x{}'.format(i), 'y{}'.format(i) , 'z{}'.format(i), 'v{}'.format(i)]

In [ ]:
with open(filename, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks_class)

In [ ]:
class_name = "airsquat"

In [ ]:
path = './gestures'

    
dir = path
sub_dir = [name for name in os.listdir(dir) if os.path.isdir(os.path.join(dir, name))]

In [ ]:
sub_dir

['physical', 'testing']

In [ ]:
for i in sub_dir:
        class_name = i
        path_to_file = dir+'/'+i

In [ ]:
path_to_file

'./gestures/testing'

In [ ]:
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
#0 for webcam
#1 for default video
path_to_file = 'gestures/physical/airsquat.mp4'
capture = request_capture(1, path_to_file)
pTime = 0
while(1):
    ret , frame = capture.read()

    #mpPose processa cada frame no formato RGB

    try:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False


        results = holistic.process(image)


        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #R_landmark = results.pose_landmarks

        cTime = time.time()
    
        landmmark_work(results, mpPose, mpDraw, mp_holistic, frame)
    
        fps = 1/(cTime - pTime)
        pTime = cTime
    
        cv2.putText(frame, str(int(fps)), (60,60), cv2.FONT_HERSHEY_PLAIN, 3, (255,100,120),3)
        cv2.imshow("Video", frame)


        try:

            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            if(results.face_landmarks):
              face = results.face_landmarks.landmark
              face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            else:
              face_row = list(np.zeros(468*4).flatten())

            row = pose_row+face_row
        
            row.insert(0, class_name)
        
            with open(filename, mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
        
        except:
            pass
    except:
        pass


    k = cv2.waitKey(5) & 0xff
    if k == 27:
        break

capture.release()
cv2.destroyAllWindows()

Model

In [ ]:
data= pd.read_csv('coords.csv')

In [ ]:
data

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
0,pull-up,0.456191,0.531645,-0.556522,0.999973,0.447566,0.529380,-0.566028,0.999973,0.443536,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,pull-up,0.456181,0.519706,-0.619696,0.999959,0.450016,0.518875,-0.625350,0.999960,0.447758,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,pull-up,0.456565,0.513497,-0.662822,0.999962,0.451749,0.514782,-0.667248,0.999963,0.449553,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
3,pull-up,0.457030,0.511387,-0.660776,0.999965,0.452143,0.511066,-0.669083,0.999966,0.449536,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,pull-up,0.457069,0.507577,-0.635600,0.999966,0.452206,0.509146,-0.643832,0.999968,0.449553,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24253,t_pose,0.513963,0.134228,-0.252745,0.994912,0.517080,0.121628,-0.229720,0.993884,0.520571,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
24254,t_pose,0.523934,0.118418,-0.229812,0.995420,0.523464,0.103836,-0.195756,0.994495,0.525495,...,0.002717,0.0,0.526031,0.105394,0.016314,0.0,0.526323,0.103855,0.017355,0.0
24255,t_pose,0.499595,0.229441,0.193735,0.995873,0.486559,0.213212,0.167615,0.995038,0.480210,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
24256,t_pose,0.505574,0.127902,-0.303131,0.996282,0.511041,0.107321,-0.292557,0.995515,0.514666,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0


In [ ]:
data['class'].value_counts()

pull-up     8068
standing    6475
seated      4482
push-up     2724
t_pose      1352
punching    1157
Name: class, dtype: int64

In [ ]:
from sklearn import preprocessing


le = preprocessing.LabelEncoder()

In [ ]:
data['class'] = le.fit_transform(data['class'])

C:\Users\victo\AppData\Local\Temp\ipykernel_35900\1370832270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['class'] = le.fit_transform(data['class'])


In [ ]:
le.classes_

array(['salute', 't_pose'], dtype=object)

In [ ]:
n_inputs, n_outputs = X.shape[1], y.shape[1]

In [ ]:
X = data.iloc[:, 1:]
y = data.iloc[:, :1]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.1, random_state=40)

In [ ]:
from xgboost import XGBClassifier


xgb = XGBClassifier(n_estimators = 1000)

In [ ]:
xgb.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=1000,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [ ]:
predicted = xgb.predict(X_test)

In [ ]:
from sklearn.metrics import f1_score


f1_score(predicted,y_test, average='macro')

0.9362851605112147

from vid

In [ ]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

c:\Users\victo\anaconda3\envs\dev\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\victo\anaconda3\envs\dev\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\victo\anaconda3\envs\dev\lib\site-packages\sklearn\linear_model\_ridge.py:1178: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to

In [ ]:
out = fit_models['rc'].predict(X_test)

for i in fit_models:
    out = fit_models[i].predict(X_test)
    print(f1_score(out,y_test, average='macro'))

0.9835738792260814
0.9958046648654854
0.9841166205735289
0.979246917698053


Predict and Show

In [ ]:
holistic = mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5)
    
#0 for webcam
#1 for default video
path_to_file = 'gestures/testing/airsquat-wild.mp4'
capture = request_capture(1, path_to_file)
pTime = 0
while(1):
    ret , frame = capture.read()

    try:
        #mpPose processa cada frame no formato RGB
        frameRGB = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
        results = holistic.process(frameRGB)

        #R_landmark = results.pose_landmarks

        cTime = time.time()
        
        landmmark_work(results, mpPose, mpDraw, mp_holistic, frame)
        
        fps = 1/(cTime - pTime)
        pTime = cTime

        cv2.putText(frame, str(int(fps)), (60,60), cv2.FONT_HERSHEY_PLAIN, 3, (250,0,90),3)
        

        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
            # Extract Face landmarks
            if(results.face_landmarks):
                face = results.face_landmarks.landmark
                face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            else:
                face_row = list(np.zeros(468*4).flatten())
            # Concate rows
            
            row = pose_row+face_row


            
            X = np.array(row).reshape(1,-1)
            body_language_class = le.inverse_transform(xgb.predict(X))
            body_language_prob = xgb.predict_proba(X)
            #print(body_language_class, body_language_prob)

            
            # Get status box
            #cv2.rectangle(frame, (100,100), (250, 60), (245, 117, 16), -1)
            
            # Display Class
            cv2.putText(frame, 'CLASS'
                        , (120,140), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, body_language_class[0]
                        , (130,168), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 233), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(frame, 'PROB'
                        , (300,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(frame, str(round(body_language_prob.max(),2))
                        , (300,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 233), 2, cv2.LINE_AA)
        except:
            pass

    except:
        pass

    cv2.imshow("Video", frame)

    k = cv2.waitKey(5) & 0xff
    if k == 27:
        break

capture.release()
cv2.destroyAllWindows()

In [ ]:
np.array(row).reshape(1,-1)[0]

array([ 0.51385081,  0.44113106, -1.17197299, ...,  0.37191319,
        0.03729007,  0.        ])

In [ ]:
body_language_class[0]

'cross'

In [ ]:
body_language_prob.max()

0.43646085

In [ ]:
X_test[:1]

,x1,y1,z1,v1,x2,y2,z2,v2,x3,y3,...,z499,v499,x500,y500,z500,v500,x501,y501,z501,v501
431,0.613151,0.323376,-0.863807,0.999949,0.638241,0.280118,-0.788851,0.999919,0.652348,0.282709,...,0.006362,0.0,0.649286,0.283245,0.022695,0.0,0.653343,0.280082,0.023166,0.0


In [ ]:
np.array(row).shape

(2004,)

In [ ]:
np.array(row).reshape(1,-1)

array([[ 0.57147515,  0.3097128 , -1.16323829, ...,  0.24264657,
         0.0232226 ,  0.        ]])

In [ ]:
dir = './gestures'
sub_dir = [name for name in os.listdir(dir) if os.path.isdir(os.path.join(dir, name))]

In [ ]:
sub_dir

['physical', 'testing']

In [ ]:
new_path = dir+'/'+sub_dir[0]
new_path

'./gestures/physical'

In [ ]:
files = [name for name in os.listdir(new_path) if os.path.isfile((os.path.join(new_path, name)))]
files

['air-squat.mp4',
 'airsquat.mp4',
 'fighting1.mp4',
 'fighting2.mp4',
 'jumping-jack.mp4',
 'push-up.mp4']

In [ ]:
new_path+'/'+files[0]

'./gestures/physical/air-squat.mp4'